In [ ]:
from pathlib import Path

# Path to your SKEMPI CSV used by parse_skempi()
SKEMPI_CSV = Path("../../SKEMPI_dataset_download/skempi_v2.csv")

# One or more base-* input directories that you already processed.
# Example: ["../../MT_Processing_Archive/base-7"]
BASE_PATHS = [
    "../../MT_Processing_Archive/base-0",
    "../../MT_Processing_Archive/base-1",
    "../../MT_Processing_Archive/base-2",
    "../../MT_Processing_Archive/base-3",
    "../../MT_Processing_Archive/base-4",
    "../../MT_Processing_Archive/base-5",
    "../../MT_Processing_Archive/base-6",
]

# Wildtype input directories (each contains many PDB folders with H*-* halves)
# Adjust to where your WT inputs actually are.
WT_PATHS = [
    "../../SKEMPI_dataset_download/wt",
]

# Where DiffBond actually wrote outputs (choose one): "results" or "Results"
OUTPUT_ROOT = "results"

# If you want to check for a *specific* file inside each output dir, set it here.
# e.g., EXPECTED_FILE = "combined_edges.parquet" or None to accept any non-empty dir.
EXPECTED_FILE = None  # e.g., "combined_edges.parquet"

# If you only want to audit starting at a certain folder number, set START_FROM
# to a string like "00325" or an int like 325. Or leave as None to audit all.
START_FROM = None

# CSV outputs will be written next to this script/notebook.
CSV_DIR = Path("./")
CSV_DIR.mkdir(exist_ok=True)

print("Configured:")
print("  SKEMPI_CSV:", SKEMPI_CSV)
print("  BASE_PATHS:", BASE_PATHS)
print("  WT_PATHS:", WT_PATHS)
print("  OUTPUT_ROOT:", OUTPUT_ROOT)
print("  EXPECTED_FILE:", EXPECTED_FILE)
print("  START_FROM:", START_FROM)
print("  CSV_DIR:", CSV_DIR.resolve())

Configured:
  SKEMPI_CSV: ../../SKEMPI_dataset_download/skempi_v2.csv
  BASE_PATHS: ['../../MT_Processing_Archive/base-0', '../../MT_Processing_Archive/base-1', '../../MT_Processing_Archive/base-2', '../../MT_Processing_Archive/base-3', '../../MT_Processing_Archive/base-4', '../../MT_Processing_Archive/base-5', '../../MT_Processing_Archive/base-6', '../../MT_Processing_Archive/base-7']
  OUTPUT_ROOT: results
  EXPECTED_FILE: None
  START_FROM: None
  CSV_DIR: /Users/justintam/Lehigh University Dropbox/Justin Tam/Research Resources/2024_DiffBondv3/DiffBond


In [ ]:
import csv

def rebuild_idx_pdb_dict(skempi_csv: Path):
    """
    Mirrors parse_skempi():
      - Skip header row
      - base_idx starts at 0
      - base_idx increments when (index % 1000 == 0)
      - formatted_index = f"{index:05d}"
      - mapping: formatted_index -> "base-{base_idx}/{pdb}/{formatted_index}"
    """
    idx_pdb_dict = {}
    base_idx = 0
    with open(skempi_csv, "r", newline="") as f:
        reader = csv.reader(f, delimiter=";")
        for index, row in enumerate(reader):
            if index == 0:
                continue  # header
            pdb = row[0].split("_")[0]
            if index % 1000 == 0:
                base_idx += 1
            formatted_index = f"{index:05d}"
            idx_pdb_dict[formatted_index] = f"base-{base_idx}/{pdb}/{formatted_index}"
    return idx_pdb_dict

idx_pdb_dict = rebuild_idx_pdb_dict(SKIEMPI_CSV)
print("idx_pdb_dict size:", len(idx_pdb_dict))
print("sample:", list(idx_pdb_dict.items())[:3])

idx_pdb_dict size: 7085
sample: [('00001', 'base-0/1CSE/00001'), ('00002', 'base-0/1CSE/00002'), ('00003', 'base-0/1CSE/00003')]


In [ ]:
import os
from typing import Dict, List, Optional, Tuple

def _ordered_folders(path: str, start_from: Optional[object] = None) -> List[str]:
    try:
        entries = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    except OSError as e:
        print(f"[ERROR] Could not list directory '{path}': {e}")
        return []
    numeric = sorted([d for d in entries if d.isdigit()], key=lambda s: int(s))
    non_numeric = [d for d in entries if not d.isdigit()]
    if start_from is None:
        return numeric + non_numeric
    s = str(start_from)
    if s.isdigit():
        start_int = int(s)
        numeric = [d for d in numeric if int(d) >= start_int]
        return numeric + non_numeric
    else:
        ordered = numeric + non_numeric
        if s in ordered:
            return ordered[ordered.index(s):]
        print(f"[WARN] start_from '{s}' not found; processing all.")
        return ordered

def _dir_nonempty(p: Path) -> bool:
    try:
        return p.exists() and any(p.iterdir())
    except Exception:
        return False

def write_missing_csv(rows: List[Tuple[str, str]], csv_path: Path) -> None:
    import csv
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["input_id", "expected_output_dir"])
        w.writerows(rows)
    print(f"[INFO] Wrote CSV: {csv_path} ({len(rows)} rows)")


# === 🧪 Audit mutants (base-*) ===
def audit_base_path(
    base_path: str,
    idx_pdb_dict: Dict[str, str],
    output_root: str = "results",
    expected_file: Optional[str] = None,
    start_from: Optional[object] = None,
) -> Tuple[int, int, List[Tuple[str, str]]]:
    folders = _ordered_folders(base_path, start_from)
    total_inputs = sum(1 for f in folders if str(f).isdigit())
    success = 0
    missing: List[Tuple[str, str]] = []

    for folder in folders:
        if not str(folder).isdigit():
            continue
        if folder not in idx_pdb_dict:
            # Can't infer output path; record as missing mapping
            missing.append((folder, "<no mapping in idx_pdb_dict>"))
            continue
        rel = idx_pdb_dict[folder]  # e.g., base-3/1ABC/00042
        out_dir = Path(output_root) / rel
        if expected_file:
            ok = (out_dir / expected_file).is_file()
        else:
            ok = _dir_nonempty(out_dir)
        if ok:
            success += 1
        else:
            missing.append((folder, str(out_dir)))

    return total_inputs, success, missing


# === 🧪 Audit wildtypes ===
def audit_wt_path(
    wt_root: str,
    output_root: str = "results",
    expected_file: Optional[str] = None,
) -> Tuple[int, int, List[Tuple[str, str]]]:
    """
    WT outputs are written as OUTPUT_ROOT / "wt" / <PDB>
    Count a PDB as success if that directory exists and is non-empty
    (or contains EXPECTED_FILE if provided).
    """
    try:
        pdb_dirs = [d for d in os.listdir(wt_root) if os.path.isdir(os.path.join(wt_root, d))]
    except OSError as e:
        print(f"[ERROR] Could not list WT root '{wt_root}': {e}")
        return 0, 0, []

    # Filter to plausible PDB folder names (alnum, usually length 4; keep permissive)
    pdbs = [d for d in pdb_dirs if len(d) >= 4]  # adjust if needed

    total_inputs = len(pdbs)
    success = 0
    missing: List[Tuple[str, str]] = []

    for pdb in sorted(pdbs):
        out_dir = Path(output_root) / "wt" / pdb
        if expected_file:
            ok = (out_dir / expected_file).is_file()
        else:
            ok = _dir_nonempty(out_dir)
        if ok:
            success += 1
        else:
            missing.append((pdb, str(out_dir)))

    return total_inputs, success, missing

In [ ]:
# === ▶️ Run: mutants ===
mutant_summary = []
for base_path in BASE_PATHS:
    print(f"\n=== Auditing mutants: {base_path} ===")
    total, success, missing = audit_base_path(
        base_path=base_path,
        idx_pdb_dict=idx_pdb_dict,
        output_root=OUTPUT_ROOT,
        expected_file=EXPECTED_FILE,
        start_from=START_FROM,
    )
    failed = total - success
    rate = (success / total * 100.0) if total else 0.0
    print(f"inputs={total} | success={success} | failed={failed} | success_rate={rate:.2f}%")

    csv_path = CSV_DIR / f"audit_missing_{Path(base_path).name}.csv"
    write_missing_csv(missing, csv_path)
    mutant_summary.append((base_path, total, success, failed, rate))

print("\n=== Mutant Summary ===")
for base_path, total, success, failed, rate in mutant_summary:
    print(f"{base_path}: inputs={total}, success={success}, failed={failed}, success_rate={rate:.2f}%")


# === ▶️ Run: wildtypes ===
wt_summary = []
all_wt_missing: List[Tuple[str, str]] = []
for wt_path in WT_PATHS:
    print(f"\n=== Auditing wildtypes: {wt_path} ===")
    total, success, missing = audit_wt_path(
        wt_root=wt_path,
        output_root=OUTPUT_ROOT,
        expected_file=EXPECTED_FILE,
    )
    failed = total - success
    rate = (success / total * 100.0) if total else 0.0
    print(f"WT inputs={total} | success={success} | failed={failed} | success_rate={rate:.2f}%")

    csv_path = CSV_DIR / f"audit_missing_wt_{Path(wt_path).name}.csv"
    write_missing_csv(missing, csv_path)
    wt_summary.append((wt_path, total, success, failed, rate))
    all_wt_missing.extend(missing)

print("\n=== WT Summary ===")
for wt_path, total, success, failed, rate in wt_summary:
    print(f"{wt_path}: inputs={total}, success={success}, failed={failed}, success_rate={rate:.2f}%")

# Optional: combined WT CSV
if len(WT_PATHS) > 1:
    combined_csv = CSV_DIR / "audit_missing_wt_all.csv"
    write_missing_csv(all_wt_missing, combined_csv)


=== Auditing: ../../MT_Processing_Archive/base-0 ===
[ERROR] Could not list directory '../../MT_Processing_Archive/base-0': [Errno 2] No such file or directory: '../../MT_Processing_Archive/base-0'
inputs=0 | success=0 | failed=0 | success_rate=0.00%
[INFO] Wrote missing CSV: audit_missing_base-0.csv (0 rows)

=== Auditing: ../../MT_Processing_Archive/base-1 ===
[ERROR] Could not list directory '../../MT_Processing_Archive/base-1': [Errno 2] No such file or directory: '../../MT_Processing_Archive/base-1'
inputs=0 | success=0 | failed=0 | success_rate=0.00%
[INFO] Wrote missing CSV: audit_missing_base-1.csv (0 rows)

=== Auditing: ../../MT_Processing_Archive/base-2 ===
[ERROR] Could not list directory '../../MT_Processing_Archive/base-2': [Errno 2] No such file or directory: '../../MT_Processing_Archive/base-2'
inputs=0 | success=0 | failed=0 | success_rate=0.00%
[INFO] Wrote missing CSV: audit_missing_base-2.csv (0 rows)

=== Auditing: ../../MT_Processing_Archive/base-3 ===
[ERROR] Co